<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Skriptsprachen
### Sommersemester 2021
Prof. Dr. Heiner Giefers

## Modularisierung

Als Modul bezeichnete man in Python eine Datei mit Definitionen und Anweisungen. Jedes Python Script ist somit auch ein Modul. In Module kapselt man für gewöhnlich Code, der einem bestimmten Zweck dient. Module können andere Module importieren und somit auf deren Funktionalität zugreifen.

Dabei unterscheidet man in Python *globale* und *lokale* Module, wobei der einzige Unterschied darin besteht, wo die Dateien im System abgelegt sind. Lokale Module befinden sich im Verzeichnisbaum des Hauptprogramms, globale Module (auch Bibliotheken genannt) sind systemweit installiert und werden über den sogenannten Suchpfad gefunden.

Aus welchen Dateisystempfaden sich der Suchpfad zusammensetzt, hängt von Ihrem System ab. Sie können die voreingestellten Pfade durch setzen der Umgebungsvariablen `PYTHONPATH` erweitern.

In [ ]:
!echo $PYTHONPATH  #Linux
!echo %PYTHONPATH% #Windows

Mit `!` können Kommandos eingeleitet werden, die auf der System-Shell ausgeführt werden sollen.

Das Sonderzeichen gehört zu den sogenannten *magic commands* der interaktiven *ipython* Umgebung (die auch in Jupyter verwendet wird).
Fast alle dieser Kommandos werden mit einem `%` eingeleitet.
Eine Liste aller verfügbaren magic commands bekommand über `%lsmagic`:

In [ ]:
%lsmagic

Ein Beispiel für ein *magic command* ist `%time`.
Damit kann die Laufzeit einer Python Anweisung gemessen werden.

In [ ]:
def harm_reihe(n):
    x=0
    for i in range(1,n+1): x+=1/i
    return x

%time harm_reihe(100000)

Die Laufzeiten können allerdings stark schwanken.
Es gibt mit `timeit` eine weitere Funktion, die die Anweisung mehrfach wiederholt und den Mittelwert der Laufzeiten berechnet.

In [ ]:
%timeit harm_reihe(100000)

Um den kompletten Suchpfad auszugeben, können Sie sich die Liste `path` aus dem Modul `sys` anzeigen lassen:

In [ ]:
import sys
for path in sys.path:
    print(path)

Wie Sie am besten ihre eigenen Module zum Suchpfad hinzufügen hängt auch mit der Python-Installation zusammen.

Auf Linux und dem Standard *CPython* ist der `PYTHONPATH` eine gute Wahl.
Bei *Anaconda* gibt es ein kleines Werkzeug namens `conda-develop`, mit dem eigene Module zum Suchpfad hinzugefügt werden können.

In [ ]:
!conda-develop --help

In [ ]:
!conda-develop "~\dev\python_modules"
#!conda-develop -u "C:\Users\hgief\dev\python_modules"
## RESTART KERNEL!

Sobald Sie ein Modul importiert haben, können Sie sich Informationen über das Modul anzeigen lassen. Verwenden Sie dazu z.B. folgende Funktion:

In [ ]:
def modul_info(*mypackages):
    for p in mypackages:
        try:
            p = __import__(p)
            fs = "Modulname: {}  ---  Dateipfad: {}"
            print( fs.format(p.__name__, p.__file__))
        except Exception:
            print("Fehler bei Modul: %s" % p)

In [ ]:
modul_info("site")
modul_info("math")
modul_info("sys")

Diese Funktion verwendet die Modul-internen Referenzen `__name__` und `__file__`. Der Aufruf `modul_info("sys")` erzeugt eine Fehlermeldung, da `sys` ein eingebautes Modul ist und damit fest in den (benutzten) Python Interpreter kompiliert ist.  

Eine Liste der eingebauten Module finden Sie in `sys.builtin_module_names`.

In [ ]:
import sys
print(sys.builtin_module_names)


Neben den eingebauten Modulen gibt es noch eine Reihe weiterer Module die (normalerweise) in jeder Python Distribution zur Verfügung stehen. Diese Zusammenfassung von Modulen nennt man auch Standardbibliothek. Das Lehrbuch gibt in Teil IV einen Einblick in einige Module der Standardbibliothek.

### Einbinden von Modulen

Module werden mittels der `import` Anweisung eingebunden. ```import math``` z.B., erlaubt den Zugriff auf wichtige mathematische Funktionen und Konstanten:

In [ ]:
import math
print("Pi = %s" % math.pi)
print("Die Quadratwurzel auf 16 ist %s" % math.sqrt(16))

Wenn Sie ein Modul auf o.g. Weise importieren wird ein neuer Namensraum erzeugt, über den Sie Zugriff auf die Inhalte des Moduls bekommen. Ohne die Angabe des Namensraumes können lokale Referenzen des eingebundenen Moduls vom Interpreter nicht aufgelöst werden:

In [ ]:
import math
sqrt(16)

Falls Ihnen die Schreibweise mit dem vorangestellten Namensraum zu lang ist, können Sie den Namensraum mit dem Zusatz `as` umbenennen:

In [ ]:
import math as m
m.sqrt(16)

Mithilfe der `from`-Anweisung können aber auch Referenzen aus einem Modul in den lokalen Namensraum übernommen werden. 
Es gibt mehrere Möglichkeiten `from` im Zusammenhang mit `import` zu nutzen.  
Sie können alle Referenzen des importierten Moduls mit dem `*`-Operator einbinden:

In [ ]:
#Die nächsten 2 Zeilen bewirken, dass die zuvor importierten Referenzen aus dem Namensraum gelöscht werden
try: del pi, sqrt, math
except: pass

import math
from math import *
sqrt(pi)

Der Ausdruck `del pi, sqrt, math` funktioniert nur korrekt, wenn alle Namen auch definiert sind. Beim ersten undefinierten Namen bricht der try/except block ab und belässt ggf. existierende weitere Namen definiert. Daher bietet es sich an, eine Funktion zu entwickeln, die alle Namen durchläuft und alle existierende Definitionen löscht:

In [ ]:
def del_tokens(*tnames):
    for mname in tnames:
        try:
            del globals()[mname]
            print("Deleted reference %s" % mname)
        except: pass

Allerdings sollten Sie es möglichst vermeiden, alle Referenzen eines Moduls zu importieren (wenn Sie nur einen Teil benutzen wollen). Sie könnten damit, ohne es zu beabsichtigen, bestehende lokale Referenzen überschreiben: 

In [ ]:
del_tokens("pi", "sqrt", "math")

def sqrt(n):
    return 42

from math import *

sqrt(4)


Es ist daher ratsam, nur diejenigen Referenzen einzubinden, die Sie benötigen:

In [ ]:
del_tokens("pi", "sqrt", "math")

import math
from math import sqrt
sqrt(pi) 

Übrigens können Sie auch hier den Namen der importierten Referenz überschreiben:

In [ ]:
del_tokens("pi", "sqrt", "math")
from math import sqrt as wurzel
wurzel(16)

### Eigene Module Definieren
Die einfachste Art der Modularisierung in Python sind lokale Module. Sie werden implementiert, indem man einzelne Programmteile in Dateien kapselt, die sich im Verzeichnis (oder in Unterverzeichnissen) des Hauptprogramms befinden.

#### Aufgabe 1
**Erstellen Sie ein Modul "mystrangemath.py" im Verzeichnis Ihres Hauptprogramms. Das Modul soll eine Funktion `sqrt(n)` enthalten, die, statt der Quadratwurzel von n, eine beliebige Konstante zurück gibt. Schreiben Sie einen Test in dem die "echte", sowie die fehlerhafte Version der `sqrt`-Funktion benutzt werden.**

Hinweise: Wenn Sie auf Ihrem lokalen Computer arbeiten, können Sie die Verzeichnisse und Dateien auf "normalem Weg" (Dateibrowser, Shell, Editor...) erstellen und editieren. Falls das nicht möglich ist, weil Sie z.B. auf einem Notebook-Server arbeiten und keinen expliziten Zugriff auf Ihre Dateien haben, können Sie shell-Kommandos innerhalb von ipython/Jupyter nutzen. Mit `!cmd` können Sie eine Kommando `cmd` in der shell ausführen. Der folgende Code-Abschnitt enthält einige Beispiele:

In [ ]:
#Gibt den aktuellen Verzeichnispfad aus
!pwd
#Löscht die Datei test.py aus dem aktuellen Verzeichnis
!rm test.py

Den Inhalt einer lokalen Datei zu schreiben können Sie z.B. mittels der Datei Ein-/Ausgabe in Python realisieren:

In [ ]:
c = """Dies is
ein mehrzeiliger Text,
    mit Einrückungen!"""
f = open('./test.py', 'w')
f.write(c)
f.close()
!cat test.py
!rm test.py

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
from mystrangemath import sqrt
assert sqrt(4) == sqrt(9) == sqrt(100), 'sqrt function should return an arbitrary constant instead of the square root of n'
del_tokens("sqrt")

In [ ]:
from mystrangemath import sqrt as sqrt_falsch
from math import sqrt
print("Die Quadratwurzel auf 16 ist %s und nicht %s" % (sqrt(16), sqrt_falsch(16)))

### Pakete
Sie können ein oder mehrere Module zu einem Paket zusammenfassen. Um ein Paket zu erstellen, muss ein Unterordner im Programmverzeichnis erzeugt werden, welcher eine Datei namens `__init__.py` enthält. Der Name des Ordners entspricht dem Namen des Pakets.

#### Aufgabe 2
**Erstellen Sie ein Paket welches das Modul "mystrangemath.py" enthält. Testen Sie Ihre Implementierung indem Sie das Paket importieren und eine Funktion daraus Aufrufen.**

In [ ]:
#Diese Aufgabe kann, aber muss nicht über das Notebook realisiert werden.

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
import os
assert os.path.isdir('./meinPaket'), 'meinPaket package does not exist!\nUse shell command to create a directory.'
assert os.path.isfile('./meinPaket/mystrangemath.py'), 'mystrangemath module is not moved!\nUse shell command to move the file.'

In [ ]:
c = "from . import mystrangemath"
f = open('./meinPaket/__init__.py', 'w')
f.write(c)
f.write("")
f.close()

In [ ]:
del_tokens("mystrangemath", "sqrt")

from meinPaket import *
mystrangemath.sqrt(16)

In [ ]:
from meinPaket.mystrangemath import sqrt
sqrt.__module__

Beachten Sie, dass Python `.py` Skripte bei erstmalige Ausführung Bytecode compiliert.

Wenn Sie die Python-Dateien eines Moduls ändern, während es an anderer Stelle eingebunden ist, werden dort die Änderungen ggf. nicht wirksam.

In [ ]:
c = """def testprint():
    print("Dies ist ein Text")"""
f = open('./dummy.py', 'w')
f.write(c)
f.close()
!cat C:\Users\hgief\dev\python_modules\dummy.py

In [ ]:
import dummy
dummy.testprint()

Mit dem *magic command* `autoload` können die das Neuladen eines Moduls erzwingen. 

In [ ]:
%load_ext autoreload
%autoreload 2
c = """def testprint():
    print("Dies ist ein Text")"""
f = open('C:\\Users\\hgief\\dev\\python_modules\\dummy.py', 'w')
f.write(c)
f.close()
!cat C:\Users\hgief\dev\python_modules\dummy.py

In [ ]:
import dummy
dummy.testprint()